### 常用名词

• host: the CPU

• device: the GPU

• host memory: the system main memory

• device memory: onboard memory on a GPU card

• kernels: 被主机执行，在设备中计算

• device function: 设备函数，只能被kernel函数调用或被其他设备函数调用

• blockspergrid：线程块数

• threadsperblock：线程数，每个线程块的线程有共享内存，基本互相不会影响速度，所以要设置的尽可能大，只要不会超内存就行，一般设为128、256、512、1024等
    ** 这两个参数都可以设置为单个整数或维度为1、2、3的tupple

### 计算当前线程的位置：
    cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
    当前线程数+线程块数*线程块的维度

### 直接返回绝对位置：
    numba.cuda.grid(ndim)： 返回当前位置在所有线程块的绝对位置，ndim为xyz的个数
    numba.cuda.gridsize(ndim)：返回其维度

### 内存控制：
    numba.cuda.device_array(shape, dtype=np.float_, strides=None, order='C', stream=0)：声明一个空向量
    numba.cuda.device_array_like(ary, stream=0)：同上
    numba.cuda.to_device(obj, stream=0, copy=True, to=None)：返回一个copy到设备的变量
    numba.copy_to_host()：返回一个copy到主机的变量
    numba.cuda.as_cuda_array(obj, sync=True):不复制数据，创造一个DeviceNDArray对象
    numba.cuda.is_cuda_array(obj)：检测该对象是否被定义为__cuda_array_interface__

### 设备向量
    类：class numba.cuda.cudadrv.devicearray.DeviceNDArray(shape, strides, dtype, stream=0, gpu_data=None)
    方法：copy_to_host(ary=None, stream=0)
        is_c_contiguous()：判断向量是否行存储在内存中的地址是连续的
        is_f_contiguous()：列
        ravel(order='C', stream=0)：展平处理
        reshape(*newshape, **kws)